# Exctract Historic Market Cap data
* [Coingecko python code](https://github.com/man-c/pycoingecko)

In [4]:
'''
    WARNING CONTROL to display or ignore all warnings
'''
import warnings; warnings.simplefilter('ignore')     #switch betweeb 'default' and 'ignore'
import traceback

''' Set debug flag to view extended error messages; else set it to False to turn off debugging mode '''
debug = True

## Initialize classes

In [104]:
import os
import sys
import configparser
from datetime import datetime, date, timedelta

sys.path.insert(1,"/home/nuwan/workspace/rezaware/")
import rezaware as reza
from wrangler.modules.assets.etl import cryptoMCExtractor as crypto
# from utils.modules.etl.load import noSQLwls as nosql
from utils.modules.etl.load import sparkNoSQLwls as nosql

''' restart initiate classes '''
if debug:
    import importlib
    reza = importlib.reload(reza)
    crypto = importlib.reload(crypto)
    nosql = importlib.reload(nosql)

clsNoSQL = nosql.NoSQLWorkLoads(desc="get crypto macket capitalization data")
''' optional - if not specified class will use the default values '''
prop_kwargs = {"WRITE_TO_TMP":True,   # necessary to emulate the etl dag
              }
clsMC = crypto.CryptoMarkets(
    desc='get crypto macket capitalization data', **prop_kwargs)
print("\nClass initialization and load complete!")

All python packages in rezaware loaded successfully!
All assets-module etl-packages in function-CryptoMarket imported successfully!
All etl-module load-packages in function-mongodbwls imported successfully!
mongodbwls Class initialization complete
mongodbwls Class initialization complete
CryptoMarket Class initialization complete

Class initialization and load complete!


## Read catalog data
* Distinct API categories: "asset.metadata", "historic.prices", "latest.prices"

In [107]:
from datetime import datetime
from bson.json_util import dumps

__now_dt__ = str(datetime.utcnow().isoformat())
__as_type__ = 'dict'
__database__ = "tip-historic-marketcap"
__collection__ = "coingecko.asset.list"
__find__ = {"asset.id":{"$in":['01coin']},
             "source":{"$eq":"coingecko"},
           }

clsNoSQL.connect={'DBAUTHSOURCE':__database__}
_data = clsNoSQL.read_documents(
    as_type=__as_type__,
    db_name = __database__,
    db_coll=__collection__,
    doc_find=None#__find__
)
print(dumps(_data, indent=2))

[
  {
    "_id": {
      "$oid": "6395617a6c7ec3fa7086e2cf"
    },
    "source": "coingecko",
    "asset": {
      "isactive": 0.0
    }
  },
  {
    "_id": {
      "$oid": "639561926c7ec3fa7086e2d7"
    },
    "source": "coingecko",
    "asset": {
      "isactive": 0.0
    }
  }
]


## Extract Historic Marketcap

In [14]:
import time

_data_owner='coingecko'
kwargs = {}

# kwargs = {"ASSETS":[
#     {
#     "_id": {
#       "$oid": "638740c7d8ab1a48899bf0a5"
#     },
#     "source": "coingecko",
#     "name": "Bitcoin",
#     "symbol": "btc",
#     "lastupdated": {
#       "$date": "2022-11-30T19:43:19.855Z"
#     },
#     "asset.id": "bitcoin",
#     "asset.isactive": 0,
#     "asset.tokenaddress": '',
#     "asset.platforms": {},
#     "asset": {
#       "id": "bitcoin",
#       "isactive": 1.0,
#       "platforms": {},
#       "tokenaddress": ''
#     }
#   }]
#          }

_from_dt=date(2021,10,1)
_to_dt=date(2021,12,31)

_results=None
_results, _failed = clsMC.extract_historic_mcap(
    data_owner=_data_owner,
    from_date = _from_dt,
    to_date = _to_dt,
    **kwargs,
)
if _results is None:
    print("No results but process completed")
else:
    print("Process completed with %d historic extractions" %len(_results))

Now processing COINGECKO from 2021-10-01 to 2021-12-31


KeyboardInterrupt: 

## Direct DB update datasource

In [ ]:
db.coingecko.asset.list.updateMany(
   {
       "source" :  {"$eq":'coingecko'},
       "asset.id":{"$in":['safemoon-2', 'safutitano', 'samsunspor-fan-token', 'santos-fc-fan-token', 'sao-paulo-fc-fan-token', 'savage', 'savanna', 'save-baby-doge', 'scarecrow', 's-c-corinthians-fan-token', 'seba', 'seeds', 'sekuya', 'shibamax', 'shoot', 'shuey-rhon-inu', 'silva-token', 'smartlink', 'snovio', 'solana-ecosystem-index', 'solape-token', 'solcial', 'solidex', 'solootbox-dao', 'songbird', 'spacecowboy', 'spacerat', 'sphere-finance', 'spiritdao-ghost', 'sporkdao', 'stacktical', 'stader', 'starbots', 'stargaze', 'stobox-token', 'ston', 'straitsx-indonesia-rupiah', 'streamr', 'sugarbounce', 'swapxi-token', 'talecraft', 'team-vitality-fan-token', 'tectonic', 'temple', 'tethys-finance', 'the-crypto-you', 'the-dynasty', 'thetan-coin', 'thorus', 'timeleap-finance', 'tip-blue', 'tokenplace', 'toko', 'toucan-protocol-base-carbon-tonne', 'tradao', 'transhuman-coin', 'transient', 'travel-care-2', 'treasure-sl', 'tzbtc', 'ufc-fan-token', 'unbound-finance', 'unique-utility-token', 'unitedcrowd', 'universidad-de-chile-fan-token', 'uxd-stablecoin', 'value-finance', 'varen', 'vcgamers', 'vedao', 'vee-finance', 'veraone', 'verve', 'vesta-finance', 'vicewrld', 'visor', 'vitall-markets', 'wagmi', 'wagmi-on-solana', 'wam', 'wasder', 'weentar', 'wegro', 'we-love-gm', 'whole-network', 'wicrypt', 'world-of-defish', 'wrapped-usdt', 'xbn-community-token', 'x-consoles', 'xmooney', 'xp-network', 'xtblock-token', 'xy-finance', 'yoshi-exchange', 'zenlink-network-token', 'zignaly', 'ziot', 'zone']},
   },
   [
     { "$set": 
      { "asset.isactive": 0.0 },
     }
   ],
    {
        "upsert":true
    }
)

In [ ]:
db.coingecko.asset.list.updateMany(
   {
       "source" :  {"$eq":'coingecko'},
       "asset.isactive": {"$eq":-1.0},
   },
   [
     { "$set": 
      { "asset.isactive": 1.0 },
     }
   ],
    {
        "upsert":true
    }
)

In [ ]:
db.marketcap.api.update(
   { "category" :  'crypto.metadata'},
   [
     { "$set": 
      { "api.parameters.localization": "" },
     }
   ],
    {
        "upsert":true
    }
)

## Insert/Update new datasource

In [47]:
__data__ = [
## CMC
#     {
#         "owner":'coinmarketcap',
#         "category":'historic.prices',
#         "resource.info":'https://coinmarketcap.com/api/documentation/v1/#tag/cryptocurrency',
#         "api.url":'https://pro-api.coinmarketcap.com/v1/cryptocurrency/quotes/historical',
#         "api.endpoint":'/cryptocurrency/quotes/historical',
#         "api.headers.accepts":'application/json',
#         "api.headers.method":'',
#         "api.headers.key":'',
#         "api.headers.X-CMC_PRO_API_KEY":'2f2845f0-581b-46f7-9a27-51d507826675',
#         "api.parameters.localization" :'false',
#         "api.parameters.date" :'{date}',
#         "api.parameters.cryptocurrency_type" :'all',
#         "api.parameters.start" :'1',
#         "api.parameters.limit":'5000',
#         "api.parameters.convert":'USD',
#         "api.parameters.sort":'symbol',
#     },
#     {
#         "owner":'coinmarketcap',
#         "category":'latest.prices',
#         "resource.info":'https://coinmarketcap.com/api/documentation/v1/#tag/cryptocurrency',
#         "api.url":'https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest',
#         "api.endpoint":'/cryptocurrency/*',
#         "api.headers.accepts":'application/json',
#         "api.headers.method":'',
#         "api.headers.key":'',
#         "api.headers.X-CMC_PRO_API_KEY":'2f2845f0-581b-46f7-9a27-51d507826675',
#         "api.parameters.localization" :'false',
#         "api.parameters.start" :'1',
#         "api.parameters.limit":'5000',
#         "api.parameters.convert":'USD',
#         "api.parameters.sort":'symbol',
#     },
#     {
#         "owner":'coinmarketcap',
#         "category":'asset.metadata',
#         "resource.info":'https://coinmarketcap.com/api/documentation/v1/#tag/cryptocurrency',
#         "api.url":'https://pro-api.coinmarketcap.com/v1/cryptocurrency/map',
#         "api.endpoint":'/api/v3/coins/list',
#         "api.headers.accepts":'application/json',
#         "api.headers.method":'',
#         "api.headers.X-CMC_PRO_API_KEY":'2f2845f0-581b-46f7-9a27-51d507826675',
#         "api.parameters.localization" :'',
#         "api.parameters.listing_status" :'active',
#         "api.parameters.start" :'1',
#         "api.parameters.limit":'5000',
#         "api.parameters.sort":'cmc_rank',
#     },
## COINGEKO
#     {
#         "owner":'coingecko',
#         "category":'historic.prices',
#         "resource.info":'https://www.coingecko.com/en/api/documentation',
#         "api.url":'https://api.coingecko.com/api/v3/coins/{id}/market_chart/range',
#         "api.endpoint":'/coins/{id}/history',
#         "api.headers.accepts":'application/json',
#         "api.headers.method":'',
#         "api.headers.key":'',
#         "api.parameters.localization" :'false',
#         "api.parameters.id" :'',
#         "api.parameters.days":'',
#         "api.parameters.vs_currency":'usd',
#         "api.parameters.interval" :'',
#     },
#     {
#         "owner":'coingecko',
#         "category":'latest.prices',
#         "resource.info":'https://www.coingecko.com/en/api/documentation',
#         "api.url":'https://api.coingecko.com/api/v3/coins/markets',
#         "api.endpoint":'/coins/markets',
#         "api.headers.accepts":'application/json',
#         "api.headers.method":'',
#         "api.headers.key":'',
#         "api.parameters.localization" :'false',
#         "api.parameters.per_page" :'250',
#         "api.parameters.page" :'1',
#         "api.parameters.price_change_percentage" :'1h,24h,7d,14d,30d',
#         "api.parameters.order":'market_cap_desc',
#         "api.parameters.vs_currency":'USD',
#         "api.parameters.sparkline" :'false',
#         "api.parameters.sort":'symbol',
#     },
#     {
#         "owner":'coingecko',
#         "category":'asset.metadata',
#         "resource.info":'https://www.coingecko.com/en/api/documentation',
#         "api.url":'https://api.coingecko.com/api/v3/coins/list',
#         "api.endpoint":'/coins/list',
#         "api.headers.accepts":'application/json',
#         "api.headers.method":'',
#         "api.headers.key":'',
#         "api.parameters.include_platform" :'true',
#     },
]
_uids = ['owner','category']
_database = "tip-data-sources"
_collection = "marketcap.api"

# __keys__ = []
clsNoSQL.connect={'DBAUTHSOURCE':_database}
_collection = clsNoSQL.write_documents(
    db_name=_database,
    db_coll=_collection,
    data=__data__,
    uuid_list=_uids
)
# print([x for x in _collection])

Total 1 documents, duccessful insert count = 0 & modify count = 1


In [45]:
import re
logpath = os.path.join("/home/nuwan/workspace/rezaware/",
                     "wrangler/logs/assets/etl/app.log")
with open(logpath,'r') as f:
    symbol_list=[]
    for line in f:
        if line.startswith("[WARNING]"):
            m = re.search("etl - (.+?) marketcap", line)
            if m:
                found = m.group(1)
                symbol_list.append(found.replace("'",""))
print(symbol_list)

['blocksport', 'bobatama', 'bologna-fc-fan-token', 'bomb-money', 'bone-shibaswap', 'boneswap', 'bone-token', 'bontecoin', 'boost', 'brazil-fan-token', 'brick', 'brig-finance', 'bright-token', 'btmiracles', 'btrips', 'bulldog-billionaires', 'burningmoon', 'buying', 'c2x-2', 'caashcow', 'caietf-finance', 'capital-dao-starter-token', 'carbon-credit', 'care-coin', 'cash-tech', 'castweet', 'catbonk', 'ccomp', 'centurion', 'centurion-inu', 'chainlink-wormhole', 'chainpay', 'channels', 'checkdot', 'cherryswap', 'chickenlegs', 'cirus', 'cleardao', 'clover', 'cockapoo', 'cocktailbar', 'codi-finance', 'cogiverse', 'coinary-token', 'coin-fast-alert', 'coinmerge', 'coinweb', 'coinwind', 'collector-coin', 'comfy', 'community-coin-2', 'community-vote-power', 'compound-chainlink-token', 'concertvr', 'constitutiondao-wormhole', 'coreum', 'corgidoge', 'corgi-inu', 'corginftgame', 'corn', 'cosplay-token-2', 'cougar-token', 'covicoin', 'cowry', 'cpucoin', 'cre8r-dao', 'credefi', 'creditcoin-2', 'cronosph

In [35]:
import json

logpath = os.path.join("/home/nuwan/workspace/rezaware/",
                     "wrangler/notebooks/assets/etl/crypto/nft_list.json")
nft_dict=[]
with open(logpath,'r') as f:
    nft_list = json.load(f)
nft_symb_list = [x['symbol'] for x in nft_list]
print(nft_symb_list)

['KONGZ', 'MZ', 'SYL', 'GAIA', 'SYL', 'BOMUL', 'MIYA', 'SYL', 'SW', 'GAIA', 'BELLYGOM', 'SNKRZ', 'etnah', 'MTDZ', 'DOD', 'Puuvilla', 'GAIA', 'SWC', 'kongz', 'MATE', 'K3K', 'COGO', 'UKKIKKI', 'ArcheWorld_Land', 'SMILEMIYA', 'KDGE', 'VGN', 'TREASURES', 'ArcheWorld_FandomCard', 'SE7EN', 'HT', 'MTM', 'DHB', 'RBL', 'CELPI', 'GAIA', 'GRA', 'chikn', 'BIRDIE CC', 'NRS', 'KROCK', 'LCV1', 'MTG', 'APB', 'BUMZ', 'AOZHEROES', 'KLND', 'TXP', 'FOOTAGE', 'LPV1', 'KLTW', 'UKIDS', 'CT', 'PXL', 'LEMONG 3D', 'BIRDIE', 'bayc', 'SGS', 'PUNK', 'TPC', 'larva', 'KING-KINGS', 'SNKRZ OPEN BOX', '7', 'Puuvillafriends', 'NAC', 'Born Kongz', 'WTBL', 'BK', 'DMONG', 'MEE', 'LIITA', 'DUK', 'KENOME', 'KRBC', 'ROAZ', 'EMATES', 'MAC', 'KLAPES', 'MTDZPET', 'CYECV0', 'EV', 'PG', 'psd', 'CATTP', 'Roostr', 'DHBSRP', 'TMH', 'HDEC75', 'GTU', 'SLIME', 'MAYC', 'MT', 'CMPZPFP', 'RDK', 'ALAP', 'STORKS', 'KONGZ', 'BUMZ', 'MA']


In [49]:
clsNoSQL.dbType = 'mongodb'
clsNoSQL.dbName = 'tip-historic-marketcap'
clsNoSQL.collections = {"HASINNAME":'btc'}
print(clsNoSQL.collections)

['coingecko.2022-01-01.wbtc', 'coingecko.2022-07-01.obt', 'coingecko.2022-07-01.btcmt', 'coingecko.2022-04-01.nsbt', 'coingecko.2022-01-01.nsbt', 'coingecko.2022-07-01.btc2', 'coingecko.2022-01-01.bto', 'coingecko.2022-07-01.ibbtc', 'coingecko.2022-01-01.btcred', 'coingecko.2022-01-01.btse', 'coingecko.2022-01-01.btcz', 'coingecko.2022-07-01.btrfly', 'coingecko.2022-01-01.btc', 'coingecko.2022-04-01.btg', 'coingecko.2022-04-01.renbtc', 'coingecko.2022-01-01.btu', 'coingecko.2022-04-01.btx', 'coingecko.2022-04-01.btl', 'coingecko.2022-07-01.wbtc', 'coingecko.2022-04-01.obtc', 'coingecko.2022-07-01.btnt', 'coingecko.2022-07-01.xbtx', 'coingecko.2022-01-01.btcp', 'coingecko.2022-04-01.wbtc', 'coingecko.2022-01-01.hbtc', 'coingecko.2022-07-01.btc', 'coingecko.2022-04-01.xbtx', 'coingecko.2022-01-01.hbt', 'coingecko.2022-04-01.0xbtc', 'coingecko.2022-07-01.btc2x-fli', 'coingecko.2022-01-01.obt', 'coingecko.2022-01-01.yfbt', 'coingecko.2022-07-01.btb', 'coingecko.2022-07-01.btcst', 'coingeck